In [1]:
import pandas as pd
import numpy as np

import matplotlib.pyplot as plt
from datetime import datetime

In [2]:
# daily Hoboken case data from city alerts
f = open('hoboken/hoboken.txt', 'rb')
txt = f.readlines()
f.close()

In [3]:
lst = [i.decode('utf-8') for i in txt]
lst = [i for i in lst if i != '\r\n']

In [4]:
df = pd.DataFrame(lst, columns=['raw'])
df.head()

,raw
0,May 3: 0\r\n
1,May 4: 9\r\n
2,May 5: 6\r\n
3,May 6: 11\r\n
4,May 7: 11\r\n


In [5]:
month_map = {'January':1,
            'February':2,
            'March':3,
            'April':4, 
            'May':5,
            'June':6, 
            'July':7, 
            'August':8,
            'September':9, 
            'October':10,
            'November':11,
            'December':12}


In [6]:
df['day_raw'] = df['raw'].apply(lambda x: x.split(':')[0])
df['year'] = np.where(df.index>=134, 2020, 2021)

df['month'] = df['day_raw'].apply(lambda x: x.split(' ')[0])
df['month'] = df['month'].apply(lambda x: month_map[x])
df['day'] = df['day_raw'].apply(lambda x: x.split(' ')[-1])

df['day'] = df['day'].apply(lambda x: x.replace('st',''))
df['day'] = df['day'].apply(lambda x: x.replace('nd',''))
df['day'] = df['day'].apply(lambda x: x.replace('rd',''))
df['day'] = df['day'].apply(lambda x: x.replace('th',''))


df['date'] = pd.to_datetime(df[['year','month','day']])

In [7]:
df['cases'] = df['raw'].apply(lambda x: x.split(':')[1].strip().split(' ')[0].replace('new',''))
df['cases'] = pd.to_numeric(df['cases'], errors='coerce')
df['cases'] = df['cases'].fillna(0)

In [8]:
df = df.sort_values('date', ascending=True).reset_index(drop=True)
df['cases_7'] = df['cases'].rolling(7).mean()
df['cases_100k'] = df['cases_7']*2 # 50k residents in Hoboken per last sentence

In [9]:
df.tail()

,raw,day_raw,year,month,day,date,cases,cases_7,cases_100k
434,May 10: 1\r\n,May 10,2021,5,10,2021-05-10,1.0,6.142857,12.285714
435,May 11: 0\r\n,May 11,2021,5,11,2021-05-11,0.0,4.857143,9.714286
436,May 12: 5\r\n,May 12,2021,5,12,2021-05-12,5.0,4.714286,9.428571
437,May 13: 1\r\n,May 13,2021,5,13,2021-05-13,1.0,3.285714,6.571429
438,May 14: 7\r\n,May 14,2021,5,14,2021-05-14,7.0,2.714286,5.428571


In [10]:
import pickle
pickle.dump(df, open('hoboken/hoboken.p', 'wb'))